# Business Understanding

## Problem Statement
- Our company wants to launch a movie studio but has no experience in film production. We need to identify what types of movies are currently profitable to avoid financial losses from creating unpopular films.

## Business Objectives
**Objectives**

- Identify top 5 genres with highest profit margins.

- Determine optimal release month for each genre.

- Analyze if low-budget films (<$10M) can achieve high ROI.

- Recommend 3 proven directors for hire.

## Project Goals
- Analyze movie datasets (Box Office Mojo, IMDB, etc.) to find patterns in successful films. Focus on genres, release seasons, budgets, and ratings. Create simple, clear recommendations for the studio head using basic data analysis.

## Success Criteria
- A list of film types that can be successful to the company and this list should be backed by data.

- Recommendations that are easy to understand and apply to a starting movie company.

- Analysis uses only the provided datasets.

# Data Understanding

## Data Sources Overview

The following publicly available movie industry datasets will be explored. Their collective relevance to the studio head's problem lies in their ability to answer core questions about profitable film types:

 - **Box Office Mojo** (CSV): Contains domestic gross earnings and release dates (2010-2018), directly indicating when films perform best.

 - **IMDB** (SQLite Database): Provides genre classifications, director information, and titles, essential for categorizing what types of films exist.

 - **Rotten Tomatoes** (TSV): Includes critic/audience scores, useful for understanding quality perception of successful films.

 - **TheMovieDB** (CSV): Offers popularity metrics and international revenue data, supplementing financial analysis.

 - **The Numbers** (CSV): Features production budgets and worldwide gross figures (1915-2020), enabling profitability calculations – the core success metric.

These datasets collectively address the studio head's need to identify profitable film characteristics by covering financial performance, genre trends, release timing, and creative talent

## Data Loading and Initial Exploration

### The Numbers Dataset

In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [72]:
# Load the dataset
df_budget = pd.read_csv('data/tn.movie_budgets.csv')

df_budget.head(3) 

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"


In [76]:
print(df_budget.dtypes)

id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object


important columns
- `production_budget`: Film creation costs
- `domestic_gross`: US earnings
- `worldwide_gross`: Global earnings
- `release date`

These directly measure profitability

- Financial values stored as strings(object) with currency symbols for example `$110,000,000` requiring cleaning

In [100]:
print(f'Oldest release: {df_budget['release_date'].min()}')
print(f'Newest release: {df_budget['release_date'].max()}')

Oldest release: Apr 1, 1975
Newest release: Sep 9, 2016


- The dataset covers films from `Apr 1, 1975` to `Sep 9, 2016` that is 41 years of movie financial data. This is valuable as it focuses on recent conditions which are relevant to our new studio.
- Release dates in `MM DD YYYY`

In [87]:
df_budget.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

- No missing values

**Immediate Limitations:**
- No genre information cannot yet answer "what types"
- No director/crew details
- Older films may not reflect current market dynamics

### Box Office Mojo Dataset

In [104]:
# Load the dataset
df_gross = pd.read_csv('data/bom.movie_gross.csv')
df_gross.head(3)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010


In [112]:
df_gross.dtypes

title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object

important columns:
- `Title`
- `Domestic_gross`: US box office earnings
- `Foreign_gross`: International earnings (string)
- `Year`: Release year
- `Studio`

In [110]:
df_gross.shape

(3387, 5)

In [114]:
print(f'Oldest release: {df_gross['year'].min()}')
print(f'Newest release: {df_gross['year'].max()}')

Oldest release: 2010
Newest release: 2018


- Focuses exclusively on recent films this is highly relevant for our new studio 

In [108]:
df_gross.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [126]:
df_gross.shape

(3387, 5)

- 0.826% missing values in domestic_gross, 28 films lack US earnings data
- Additional 39.85% missing in foreign_gross, 1350 films lack international data

**Note:** 
- The 2010-2018 coverage is good for the company as it aligns with current market conditions
- Missing gross values will require data cleaning
- Contains studio information

### IMDB Dataset

In [140]:
import sqlite3
# Connect to database
conn = sqlite3.connect("im.db")

In [144]:
# Check available tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables['name'].values)

[]


In [ ]:
# Load movie basics table
movie_basics = pd.read_sql("SELECT * FROM movie_basics LIMIT 5;", conn)

print(movie_basics[['movie_id', 'primary_title', 'genres']])

In [ ]:
# Check genre nulls
null_genres = pd.read_sql("SELECT COUNT(*) FROM movie_basics WHERE genres IS NULL;", conn)

print(f"Total movies: {pd.read_sql('SELECT COUNT(*) FROM movie_basics', conn).iloc[0,0]}")
print(f"Movies missing genres: {null_genres.iloc[0,0]}")

## Metadata Summary (Columns, Types, Nulls, etc.)

## Key Variables to Explore

## Preliminary Observations and Questions

# Data Preparation